In [3]:
import pandas as pd
import numpy as np


def SMA(series, window):
    return series.rolling(window).mean()

def EMA(series,window):
    alpha=2/(window+1)
    ema=[]
    for i in range(len(series)):
        if i==0:
          ema.append(series.iloc[0])
        else:
          value=alpha*series.iloc[i]+(1-alpha)*ema[i-1]
          ema.append(value)
    return pd.Series(ema,index=series.index)

def MACD(series):
    ema12 = EMA(series, 12)
    ema26 = EMA(series, 26)
    macd_line = ema12 - ema26
    signal_line = EMA(macd_line, 9)
    histogram = macd_line - signal_line

    return macd_line, signal_line, histogram

def RSI(series, window=14):
    delta = series.diff()

    gain = delta.where(delta > 0, 0.0)
    loss = -delta.where(delta < 0, 0.0)

    avg_gain = gain.rolling(window).mean()
    avg_loss = loss.rolling(window).mean()

    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))

    return rsi

def Stochastic(high, low, close, window=14):
    lowest_low = low.rolling(window).min()
    highest_high = high.rolling(window).max()

    percent_k = 100 * (close - lowest_low) / (highest_high - lowest_low)
    percent_d = percent_k.rolling(3).mean()

    return percent_k, percent_d

def Bollinger_Bands(series, window=14):
    sma = SMA(series, window)
    std = series.rolling(window).std()

    upper_band = sma + (2 * std)
    lower_band = sma - (2 * std)

    return sma, upper_band, lower_band


import yfinance as yf

reliance = yf.download("RELIANCE.NS", period="2y")
# Moving Averages
reliance["SMA_14"] = SMA(reliance["Close"], 14)
reliance["EMA_14"] = EMA(reliance["Close"], 14)

# MACD
reliance["MACD"], reliance["Signal"], reliance["Histogram"] = MACD(reliance["Close"])

# RSI
reliance["RSI"] = RSI(reliance["Close"])

# Stochastic Oscillator
reliance["%K"], reliance["%D"] = Stochastic(reliance["High"], reliance["Low"], reliance["Close"])

# Bollinger Bands
reliance["BB_Middle"], reliance["BB_Upper"], reliance["BB_Lower"] = Bollinger_Bands(reliance["Close"])

print(reliance.loc["2025-04-29"])

/tmp/ipython-input-3931194191.py:63: FutureWarning: YF.download() has changed argument auto_adjust default to True
  reliance = yf.download("RELIANCE.NS", period="2y")
[*********************100%***********************]  1 of 1 completed


Price      Ticker     
Close      RELIANCE.NS                                         1394.131958
High       RELIANCE.NS                                         1405.287477
Low        RELIANCE.NS                                         1364.550153
Open       RELIANCE.NS                                         1365.546175
Volume     RELIANCE.NS                                            32538970
SMA_14                                                         1263.759818
EMA_14                    Ticker
RELIANCE.NS    1291.450201
dtype: float64
MACD                        Ticker
RELIANCE.NS    28.618013
dtype: float64
Signal                       Ticker
RELIANCE.NS    12.87578
dtype: float64
Histogram                   Ticker
RELIANCE.NS    15.742233
dtype: float64
RSI                                                                84.1387
%K                                                               96.216832
%D                                                               94.064703
BB